In [ ]:
%matplotlib inline
import pandas as pd
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install --user 'pyarrow<=0.9'

In [ ]:
#df = pd.read_csv('../pref_pair_sets/maxrank20-12axioms.csv')
df = pd.read_parquet('../pref_pair_sets/sampled_ranks_14_axioms.parquet')
df.head(2)

In [ ]:
col_features = [x for x in df.columns if x.startswith('ax_')]
axioms = [x.replace('ax_', '') for x in col_features]
systems = list(sorted(df.system.unique()))

# Global
Predict pairwise concordance across all of the system's rankings.

In [ ]:
col_target = 'concordant'

models = {}

def run(system, penalty, C, sample_weight=None):
    res = dict(system=system, penalty=penalty, C=C, sample_weight=sample_weight)

    tmp = df[(df['system'] == system)]
    cls = LogisticRegression(penalty=penalty, C=C, solver='saga')
    xs = tmp[col_features].values
    ys = tmp[col_target]

    model_key = (system, penalty, C, sample_weight)
    
    if sample_weight == 'relative_rankdiff':
        sample_weight = tmp.rankdiff / tmp.rankdiff.max()
    elif sample_weight == 'scorediff':
        sample_weight = tmp.scorediff
    
    model = cls.fit(xs, ys, sample_weight=sample_weight)
    
    models[model_key] = model
    accuracy = model.score(xs, ys, sample_weight=sample_weight)
    res['accuracy'] = accuracy

    coef = dict(zip(col_features, model.coef_.flatten()))
    res.update(coef)
    return res

penalties = ['l1', 'l2']
Cs = [1, 0.5, 0.1, 0.01]
weightings = [None, 'relative_rankdiff', 'scorediff']
iter_count = len(systems)*len(penalties)*len(weightings)*len(Cs)

def all_runs():
    for system in systems:
        for penalty in penalties:
            for C in Cs:
                for sample_weight in weightings:
                    yield run(system, penalty, C, sample_weight)

results = pd.DataFrame(list(tqdm(all_runs(), total=iter_count)))

## Concordance prediction accuracy

In [ ]:
results.groupby('system').accuracy.describe()

In [ ]:
results.groupby(['penalty', 'C']).accuracy.describe()

In [ ]:
# kendall's tau

ktau_denominator = 20*19/2
ktau = results.copy()
def estimate_tau(r):
    k = (r.system, r.penalty, r.C, r.sample_weight)
    m = models[k]
    samples = df.query(
        'system == @r.system & concordant == 1').copy()
    xs = samples[col_features].values
    ys = samples[target].values
    preds = m.predict(xs)
    samples['pred'] = preds
    ax_scores = samples.groupby(
        ['system', 'query', 'upper_score']
    ).pred.sum().reset_index().sort_values(
        ['system', 'query', 'upper_score'], ascending=True)
    tmp = ax_scores.groupby(['system', 'query']).rank(axis=0, method='average', ascending=False)
    ax_scores[['pred_rank', 'score_rank']] = tmp
    ax_scores['concordant'] = (ax_scores.pred_rank == ax_scores.score_rank)
    ax_scores['discordant'] = ~ax_scores.concordant
    
    tmp = ax_scores.groupby(['system', 'query'])[['concordant', 'discordant']].sum()
    tmp['n'] = tmp.concordant + tmp.discordant
    tmp['tau'] = (tmp.concordant - tmp.discordant) / (tmp.n * (tmp.n - 1) / 2)
    return k, tmp.tau

for _, r in tqdm(ktau.iterrows()):
    k, tau = estimate_tau(r)
    print(k, tau.mean())



## Axiom coefficients

In [ ]:
results.head(2), 0

In [ ]:
%%time
coeffs = pd.melt(
    results, id_vars=['system', 'sample_weight', 'penalty', 'C'], 
    value_vars=col_features, var_name='axiom', value_name='coeff')
coeffs['axiom'] = coeffs.axiom.map(lambda n: n.replace('ax_', ''))

coeffs['reg'] = coeffs.apply(axis=1, func=lambda r: '{}, C={:.2f}'.format(r.penalty, r.C))


coeffs['wt'] = coeffs.sample_weight.fillna('none')#.map(lambda s: 'rank' if 'rank' in s else 'score' if 'score' in s else s)
del coeffs['sample_weight']

#coeffs = coeffs[(coeffs.penalty=='l1')]

def draw_heatmap(data, **kwargs):
    #print(data)
    data = data.pivot('system', 'axiom', 'coeff')[axioms]
    annot = data.round(2)
    sns.heatmap(data, cmap=sns.cm.vlag_r, annot=annot, vmin=-1, vmax=1, cbar=False)
    plt.yticks(rotation=0)

with sns.plotting_context(font_scale=.75):
    g = sns.FacetGrid(coeffs, col='wt', row='reg', sharex=True, sharey=True)
    g = g.map_dataframe(draw_heatmap)

plt.gcf().set_size_inches((20, 15))
plt.gcf().tight_layout()
plt.savefig('heatmaps.png', transparent=False)

In [ ]:
coeffs['axiom'].unique()
